In [ ]:
import numpy as np
import pandas as pd

class MyDecisionTree:
    def __init__(self, max_depth=None, method='gini'):
        self.max_depth = max_depth
        self.method = method
        self.tree = {}

    def cost_function(self, y):
        _, counts = np.unique(y, return_counts=True)
        proportions = counts / counts.sum()
        if self.method == 'gini':
            return 1 - np.sum(proportions ** 2)
        else:
            return -np.sum(proportions * np.log2(proportions))

    def make_split(self, X, y, feature_index, threshold):
        left_indices = X[:, feature_index] <= threshold
        right_indices = X[:, feature_index] > threshold
        X_left, y_left = X[left_indices], y[left_indices]
        X_right, y_right = X[right_indices], y[right_indices]
        return X_left, y_left, X_right, y_right

    def fit(self, X, y):
        self.tree = self._build_tree(X, y)

    def _build_tree(self, X, y, depth=0):
        if len(np.unique(y)) == 1 or depth == self.max_depth:
            return np.unique(y)[0]
        num_features = X.shape[1]
        best_feature = None
        best_threshold = None
        best_gain = 0
        best_splits = ()
        current_cost = self.cost_function(y)
        for feature_index in range(num_features):
            thresholds = np.unique(X[:, feature_index])
            for threshold in thresholds:
                splits = self.make_split(X, y, feature_index, threshold)
                X_left, y_left, X_right, y_right = splits
                if len(y_left) == 0 or len(y_right) == 0:
                    continue
                n = len(y)
                n_left, n_right = len(y_left), len(y_right)
                left_cost = self.cost_function(y_left)
                right_cost = self.cost_function(y_right)
                weighted_avg_cost = (n_left / n) * left_cost + (n_right / n) * right_cost
                gain = current_cost - weighted_avg_cost
                if gain > best_gain:
                    best_gain = gain
                    best_feature = feature_index
                    best_threshold = threshold
                    best_splits = splits
        if best_feature is None:
            return np.argmax(np.bincount(y))
        X_left, y_left, X_right, y_right = best_splits
        left_branch = self._build_tree(X_left, y_left, depth + 1)
        right_branch = self._build_tree(X_right, y_right, depth + 1)
        return (best_feature, best_threshold, left_branch, right_branch)

    def _predict(self, inputs, tree):
        if type(tree) is not tuple:
            return tree
        feature, threshold, left_branch, right_branch = tree
        if inputs[feature] <= threshold:
            return self._predict(inputs, left_branch)
        else:
            return self._predict(inputs, right_branch)

    def predict(self, X):
        predictions = [self._predict(inputs, self.tree) for inputs in X]
        return np.array(predictions)

    def score(self, X, y):
        predictions = self.predict(X)
        accuracy = np.mean(predictions == y)
        return accuracy

    def prune(self, X_val, y_val):
        self.tree = self._prune_tree(self.tree, X_val, y_val)

    def _prune_tree(self, node, X_val, y_val):
        if type(node) is not tuple:
            return node
        feature, threshold, left_branch, right_branch = node
        left_indices = X_val[:, feature] <= threshold
        right_indices = X_val[:, feature] > threshold
        if left_indices.any():
            left_branch = self._prune_tree(left_branch, X_val[left_indices], y_val[left_indices])
        if right_indices.any():
            right_branch = self._prune_tree(right_branch, X_val[right_indices], y_val[right_indices])
        accuracy_before_prune = self.score(X_val, y_val)
        if y_val.size > 0:
            most_common_class = np.argmax(np.bincount(y_val))
        else:
            most_common_class = np.argmax(np.bincount(y_val, minlength=2))  # Avoid empty bincount
        self.tree = most_common_class
        accuracy_after_prune = self.score(X_val, y_val)
        self.tree = (feature, threshold, left_branch, right_branch)
        if accuracy_after_prune >= accuracy_before_prune:
            return most_common_class
        else:
            return (feature, threshold, left_branch, right_branch)

    def _predict_all(self, tree, X):
        return [self._predict(inputs, tree) for inputs in X]



Training and evaluating on the given dataset

In [ ]:
import pandas as pd

data_path = '/Users/abhijaysingh/Documents/College/Semester 5/ML/Assignment 2/Section C/Thyroid data.csv'
thyroid_data = pd.read_csv(data_path)

thyroid_data.head()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Converting binary features to 0/1
binary_columns = thyroid_data.nunique()[thyroid_data.nunique() == 2].keys().tolist()
binary_columns = [col for col in binary_columns if col not in ['label']]
for column in binary_columns:
    thyroid_data[column] = thyroid_data[column].map({'f': 0, 't': 1})

# Encoding categorical features
categorical_columns = thyroid_data.select_dtypes(include=['object']).columns.tolist()
categorical_columns.remove('label')  # Exclude the target label from encoding

label_encoder = LabelEncoder()
for column in categorical_columns:
    thyroid_data[column] = label_encoder.fit_transform(thyroid_data[column])

# Encoding target label
thyroid_data['label'] = label_encoder.fit_transform(thyroid_data['label'])

thyroid_data['TBG'] = thyroid_data['TBG'].replace(0, thyroid_data['TBG'].mean())

# Separating features and target label
X = thyroid_data.drop('label', axis=1)
y = thyroid_data['label']

# Spliting the data into training and validation sets (80-20 split)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert to numpy arrays for our custom decision tree
X_train = X_train.to_numpy()
y_train = y_train.to_numpy()
X_val = X_val.to_numpy()
y_val = y_val.to_numpy()

X_train.shape, X_val.shape, y_train.shape, y_val.shape


In [ ]:
# Instantiating the MyDecisionTree classifier with a maximum depth
decision_tree = MyDecisionTree(max_depth=5, method='gini')

# Training the classifier on the training set
decision_tree.fit(X_train, y_train)

# Evaluating its performance on the validation set before pruning
accuracy_before_pruning = decision_tree.score(X_val, y_val)

accuracy_before_pruning


In [ ]:
decision_tree.prune(X_val, y_val)

# Evaluating its performance on the validation set after pruning
accuracy_after_pruning = decision_tree.score(X_val, y_val)

accuracy_after_pruning